# Magic: the Gathering Recommender System
___

##### Problem Statement:  
I will use data on Magic: the Gathering cards to build a content-based recommender system that suggests similar cards in order to improve card selection during the deck building process.

##### Outline:  
1. Gathering Data  
    a. The data can be gathered from Scryfall's bulk data section which has every card as a json file
2. Cleaning Data  
    a. There is a lot of unnecessary data that I can drop  
    b. Extract the nested json objects
3. EDA
4. Recommender System  
    a. Content-Based Recommender  
    b. Cosine similarity
5. Stretch Goals  
    a. Keep a running tally and rating system for a user-based collaborative recommender

##### Risks and Assumptions:  
One risk is that the data comes in the form of nested json objects which will need to be formatted in a way I can use it

##### Data Sources:  
[Scryfall Bulk Data](https://scryfall.com/docs/api/bulk-data)  
[Scryfall Default Cards](https://archive.scryfall.com/json/scryfall-default-cards.json)

## 01 - Cleaning
___

### Imports

In [2]:
import pandas as pd

pd.options.display.max_columns = 35

In [3]:
df = pd.read_json('../Data/scryfall-default-cards.json')

In [4]:
df.head()

,object,id,oracle_id,multiverse_ids,tcgplayer_id,name,lang,released_at,uri,scryfall_uri,layout,highres_image,image_uris,mana_cost,cmc,type_line,oracle_text,...,frame_effects,card_faces,promo_types,edhrec_rank,loyalty,preview,arena_id,all_parts,mtgo_id,variation_of,color_indicator,printed_name,printed_type_line,printed_text,mtgo_foil_id,life_modifier,hand_modifier
0,card,dbcdbf7a-9294-47ad-9f93-c16b78c7463a,cd6250ae-9079-4a62-8a70-0d94fbac21bc,[],200607.0,Earthshaker Giant,en,2019-11-15,https://api.scryfall.com/cards/dbcdbf7a-9294-4...,https://scryfall.com/card/gn2/5/earthshaker-gi...,normal,False,{'small': 'https://img.scryfall.com/cards/smal...,{4}{G}{G},6.0,Creature — Giant Druid,Trample\nWhen Earthshaker Giant enters the bat...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,acb3ce9b-ee4f-410a-8db3-e87aeb0a4444,ab0dfae5-b9d4-417b-8a0d-2525ae3a73b9,[],200606.0,Fiendish Duo,en,2019-11-15,https://api.scryfall.com/cards/acb3ce9b-ee4f-4...,https://scryfall.com/card/gn2/4/fiendish-duo?u...,normal,False,{'small': 'https://img.scryfall.com/cards/smal...,{4}{R}{R},6.0,Creature — Devil,First strike\nIf a source would deal damage to...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,17b2ed72-d0f0-4d8d-bb5e-dce08d157466,7264d3b3-fd46-4ea3-a85d-f0b068c331ad,[],200605.0,Calculating Lich,en,2019-11-15,https://api.scryfall.com/cards/17b2ed72-d0f0-4...,https://scryfall.com/card/gn2/3/calculating-li...,normal,False,{'small': 'https://img.scryfall.com/cards/smal...,{4}{B}{B},6.0,Creature — Zombie Wizard,Menace\nWhenever a creature attacks one of you...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0faa9eea-fbf1-41f7-9def-1ec3d5134a53,e6284fb3-7cf8-4730-b156-10085b70b0e8,[],200604.0,Sphinx of Enlightenment,en,2019-11-15,https://api.scryfall.com/cards/0faa9eea-fbf1-4...,https://scryfall.com/card/gn2/2/sphinx-of-enli...,normal,False,{'small': 'https://img.scryfall.com/cards/smal...,{4}{U}{U},6.0,Creature — Sphinx,Flying\nWhen Sphinx of Enlightenment enters th...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,ecbeac44-9392-4522-8ff5-87079386bd0a,43296f8b-58d9-446e-a538-1c4921552c41,[],200603.0,Highcliff Felidar,en,2019-11-15,https://api.scryfall.com/cards/ecbeac44-9392-4...,https://scryfall.com/card/gn2/1/highcliff-feli...,normal,False,{'small': 'https://img.scryfall.com/cards/smal...,{5}{W}{W},7.0,Creature — Cat Beast,Vigilance\nWhen Highcliff Felidar enters the b...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(48536, 71)

In [6]:
df.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'tcgplayer_id', 'name',
       'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image',
       'image_uris', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'power',
       'toughness', 'colors', 'color_identity', 'legalities', 'games',
       'reserved', 'foil', 'nonfoil', 'oversized', 'promo', 'reprint',
       'variation', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri',
       'scryfall_set_uri', 'rulings_uri', 'prints_search_uri',
       'collector_number', 'digital', 'rarity', 'flavor_text', 'card_back_id',
       'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame',
       'full_art', 'textless', 'booster', 'story_spotlight', 'related_uris',
       'watermark', 'frame_effects', 'card_faces', 'promo_types',
       'edhrec_rank', 'loyalty', 'preview', 'arena_id', 'all_parts', 'mtgo_id',
       'variation_of', 'color_indicator', 'printed_name', 'printed_type_line',
       'printed_text', 

___
### Drop unneeded columns

In [18]:
unneeded = ['id', 'oracle_id', 'multiverse_ids', 'tcgplayer_id', 'uri', 'scryfall_uri', 'image_uris', 
            'highres_image', 'games', 'set_uri', 'set_search_uri',  'scryfall_set_uri', 'rulings_uri', 
            'prints_search_uri', 'collector_number', 'card_back_id', 'artist_ids', 'illustration_id', 
            'story_spotlight', 'related_uris', 'preview', 'arena_id', 'all_parts', 'mtgo_id', 'variation_of',
            'color_indicator', 'mtgo_foil_id', 'life_modifier', 'hand_modifier', 'frame_effects', 'flavor_text',
            'watermark', 'printed_name', 'printed_type_line', 'printed_text']
df = df.drop(columns=unneeded)

In [19]:
df.head()

,object,name,lang,released_at,layout,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,legalities,reserved,foil,nonfoil,...,reprint,variation,set,set_name,set_type,digital,rarity,artist,border_color,frame,full_art,textless,booster,card_faces,promo_types,edhrec_rank,loyalty
0,card,Earthshaker Giant,en,2019-11-15,normal,{4}{G}{G},6.0,Creature — Giant Druid,Trample\nWhen Earthshaker Giant enters the bat...,6,6,[G],[G],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,False,...,False,False,gn2,Game Night 2019,box,False,mythic,Milivoj Ćeran,black,2015,False,False,False,NaN,NaN,NaN,NaN
1,card,Fiendish Duo,en,2019-11-15,normal,{4}{R}{R},6.0,Creature — Devil,First strike\nIf a source would deal damage to...,5,5,[R],[R],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,False,...,False,False,gn2,Game Night 2019,box,False,mythic,Lucas Graciano,black,2015,False,False,False,NaN,NaN,NaN,NaN
2,card,Calculating Lich,en,2019-11-15,normal,{4}{B}{B},6.0,Creature — Zombie Wizard,Menace\nWhenever a creature attacks one of you...,5,5,[B],[B],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,False,...,False,False,gn2,Game Night 2019,box,False,mythic,Antonio José Manzanedo,black,2015,False,False,False,NaN,NaN,NaN,NaN
3,card,Sphinx of Enlightenment,en,2019-11-15,normal,{4}{U}{U},6.0,Creature — Sphinx,Flying\nWhen Sphinx of Enlightenment enters th...,5,5,[U],[U],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,False,...,False,False,gn2,Game Night 2019,box,False,mythic,Johan Grenier,black,2015,False,False,False,NaN,NaN,NaN,NaN
4,card,Highcliff Felidar,en,2019-11-15,normal,{5}{W}{W},7.0,Creature — Cat Beast,Vigilance\nWhen Highcliff Felidar enters the b...,5,5,[W],[W],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,False,...,False,False,gn2,Game Night 2019,box,False,mythic,Kimonas Theodossiou,black,2015,False,False,False,NaN,NaN,NaN,NaN


In [20]:
df.columns

Index(['object', 'name', 'lang', 'released_at', 'layout', 'mana_cost', 'cmc',
       'type_line', 'oracle_text', 'power', 'toughness', 'colors',
       'color_identity', 'legalities', 'reserved', 'foil', 'nonfoil',
       'oversized', 'promo', 'reprint', 'variation', 'set', 'set_name',
       'set_type', 'digital', 'rarity', 'artist', 'border_color', 'frame',
       'full_art', 'textless', 'booster', 'card_faces', 'promo_types',
       'edhrec_rank', 'loyalty'],
      dtype='object')

I also want to drop any digital cards because I want the recommender to only look at physical cards

In [21]:
df = df.drop(df[df['digital'] == True].index)

Also drop oversized cards

In [22]:
df = df.drop(df[df['oversized'] == True].index)

In [23]:
df = df.drop(columns=['oversized', 'digital'])

In [27]:
df.loc[df['lang'] != 'en']['lang'].value_counts()

es     1203
fr      428
it      190
ja      173
de        3
ru        2
px        1
grc       1
la        1
sa        1
he        1
ar        1
Name: lang, dtype: int64

let's drop any non-english cards because not every card is available in non-english versions

In [30]:
df = df.drop(df.loc[df['lang'] != 'en'].index)

___
### check for nulls

In [31]:
df.isnull().sum()

object                0
name                  0
lang                  0
released_at           0
layout                0
mana_cost           261
cmc                   0
type_line             0
oracle_text         497
power             22575
toughness         22575
colors              261
color_identity        0
legalities            0
reserved              0
foil                  0
nonfoil               0
promo                 0
reprint               0
variation             0
set                   0
set_name              0
set_type              0
rarity                0
artist                0
border_color          0
frame                 0
full_art              0
textless              0
booster               0
card_faces        42027
promo_types       38823
edhrec_rank        4293
loyalty           42090
dtype: int64

In [32]:
df.shape

(42524, 34)

Art Series cards only existed in the modern horrizon set and are not actual cards, so we should drop them from our data set

In [33]:
df = df.drop(df[df['layout'] == 'art_series'].index)

In [34]:
# edhrec_rank nulls should be 0. meaning no decks on edhrec play the card
df['edhrec_rank'] = df['edhrec_rank'].fillna(0)

### drop tokens

In [35]:
df['layout'].value_counts()

normal                40796
token                  1048
transform               174
split                   135
adventure                74
emblem                   69
leveler                  39
double_faced_token       33
flip                     27
saga                     25
host                     19
meld                     18
augment                  13
Name: layout, dtype: int64

In [36]:
non_cards_index = df[(df['layout'] == 'double_faced_token') | (df['layout'] == 'token') | 
                             (df['layout'] == 'scheme') | (df['layout'] == 'planar') | 
                             (df['layout'] == 'vanguard') | (df['layout'] == 'emblem')].index

In [37]:
df = df.drop(non_cards_index)
df = df.drop(df[df['set_type'] == 'token'].index)
df.shape

(41262, 34)

In [38]:
df.isnull().sum()

object                0
name                  0
lang                  0
released_at           0
layout                0
mana_cost           174
cmc                   0
type_line             0
oracle_text         410
power             22247
toughness         22247
colors              174
color_identity        0
legalities            0
reserved              0
foil                  0
nonfoil               0
promo                 0
reprint               0
variation             0
set                   0
set_name              0
set_type              0
rarity                0
artist                0
border_color          0
frame                 0
full_art              0
textless              0
booster               0
card_faces        40852
promo_types       38326
edhrec_rank           0
loyalty           40828
dtype: int64

In [39]:
df[df['colors'].isnull()]['layout'].value_counts()

transform    174
Name: layout, dtype: int64

In [40]:
df[df['mana_cost'].isnull()]['layout'].value_counts()

transform    174
Name: layout, dtype: int64

In [41]:
df[df['oracle_text'].isnull()]['layout'].value_counts()

transform    174
split        135
adventure     74
flip          27
Name: layout, dtype: int64

Now I'd like to take the dual cards (transform, split, adventure, and flip cards) and break them out into their individual cards then remove the originals from the dataset

In [42]:
dual_cards = df[df['card_faces'].notnull()].copy()
dual_cards.head()

,object,name,lang,released_at,layout,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,legalities,reserved,foil,nonfoil,promo,reprint,variation,set,set_name,set_type,rarity,artist,border_color,frame,full_art,textless,booster,card_faces,promo_types,edhrec_rank,loyalty
7,card,Nightmare Moon // Princess Luna,en,2019-10-22,transform,NaN,6.0,Legendary Creature — Alicorn // Legendary Crea...,NaN,NaN,NaN,NaN,[B],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,False,True,False,False,ptg,Ponies: The Galloping,funny,mythic,John Thacker,silver,2015,False,False,False,"[{'object': 'card_face', 'name': 'Nightmare Mo...",NaN,0.0,NaN
56,card,Lovestruck Beast // Heart's Desire,en,2019-10-04,adventure,{2}{G} // {G},3.0,Creature — Beast Noble // Sorcery — Adventure,NaN,5,5,[G],[G],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,False,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Kev Walker,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Lovestruck B...","[prerelease, datestamped]",13909.0,NaN
57,card,Lovestruck Beast // Heart's Desire,en,2019-10-04,adventure,{2}{G} // {G},3.0,Creature — Beast Noble // Sorcery — Adventure,NaN,5,5,[G],[G],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,True,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Kev Walker,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Lovestruck B...",NaN,13909.0,NaN
82,card,Bonecrusher Giant // Stomp,en,2019-10-04,adventure,{2}{R} // {1}{R},3.0,Creature — Giant // Instant — Adventure,NaN,4,3,[R],[R],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,False,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Victor Adame Minguez,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Bonecrusher ...","[prerelease, datestamped]",11665.0,NaN
83,card,Bonecrusher Giant // Stomp,en,2019-10-04,adventure,{2}{R} // {1}{R},3.0,Creature — Giant // Instant — Adventure,NaN,4,3,[R],[R],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,True,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Victor Adame Minguez,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Bonecrusher ...",NaN,11665.0,NaN


In [43]:
# create another dataframe that is a seperated version of our dual cards
dual_cards_sp = pd.DataFrame()
for card in dual_cards['card_faces'].values:
    dual_cards_sp = pd.concat([dual_cards_sp, pd.DataFrame(card)], sort=False)

In [44]:
dual_cards_sp.head()

,object,name,mana_cost,type_line,oracle_text,colors,power,toughness,flavor_text,watermark,artist,artist_id,illustration_id,image_uris,color_indicator,loyalty
0,card_face,Nightmare Moon,{4}{B}{B},Legendary Creature — Alicorn,"Flying\nAs long as it's nighttime, Nightmare M...",[B],6,6,"""The night. . . will last. . .forever!""",mlpwaningmoon,John Thacker,38ee615a-e59f-4e2e-b894-2b74c6e75541,587b0f05-3512-4e2d-9569-2f5f70bc0c92,{'small': 'https://img.scryfall.com/cards/smal...,NaN,NaN
1,card_face,Princess Luna,,Legendary Creature — Alicorn,Flying\nWhen this creature transforms into Pri...,[B],4,4,NaN,mlpwaxingmoon,John Thacker,38ee615a-e59f-4e2e-b894-2b74c6e75541,a18eb5e3-0043-4d97-ba49-163db9e70df3,{'small': 'https://img.scryfall.com/cards/smal...,[B],NaN
0,card_face,Lovestruck Beast,{2}{G},Creature — Beast Noble,Lovestruck Beast can't attack unless you contr...,NaN,5,5,"His mind chose solitude, but his heart disagreed.",NaN,Kev Walker,f366a0ee-a0cd-466d-ba6a-90058c7a31a6,5313c8d4-5dc9-484d-9b1f-5349de020e4e,NaN,NaN,NaN
1,card_face,Heart's Desire,{G},Sorcery — Adventure,Create a 1/1 white Human creature token. (Then...,NaN,NaN,NaN,NaN,NaN,Kev Walker,f366a0ee-a0cd-466d-ba6a-90058c7a31a6,NaN,NaN,NaN,NaN
0,card_face,Lovestruck Beast,{2}{G},Creature — Beast Noble,Lovestruck Beast can't attack unless you contr...,NaN,5,5,"His mind chose solitude, but his heart disagreed.",NaN,Kev Walker,f366a0ee-a0cd-466d-ba6a-90058c7a31a6,5313c8d4-5dc9-484d-9b1f-5349de020e4e,NaN,NaN,NaN


In [45]:
# drop unneeded columns
drop_cols = ['watermark', 'artist_id', 'illustration_id', 'image_uris', 'color_indicator']

dual_cards_sp = dual_cards_sp.drop(columns=drop_cols)

In [47]:
df.loc[df['oracle_text'].isnull()]['set'].value_counts()

eld     60
soi     33
isd     20
emn     15
dgm     15
akh     15
peld    14
v17     13
dka     13
hou     10
grn     10
dis     10
psoi    10
pakh    10
pxln    10
rna     10
pxtc    10
chk     10
xln     10
rix      7
prix     7
pgrn     6
inv      5
sok      5
pori     5
ps15     5
ori      5
bok      5
apc      5
phou     5
plc      3
pemn     3
c19      3
ddh      3
wc02     3
wc01     3
f06      2
unh      2
hop      2
cmd      2
pisd     2
ddj      2
c16      2
pdka     2
prna     2
phuk     2
arc      1
c18      1
cm2      1
pm19     1
ptg      1
pdgm     1
m19      1
gk1      1
uma      1
tsb      1
prel     1
hho      1
c13      1
h17      1
Name: set, dtype: int64

In [ ]:
# # drop our dual cards from our main df
# df = df.drop(dual_cards.index).reset_index(drop=True)
# df.shape

In [49]:
df[df['name'] == "Lovestruck Beast // Heart's Desire"]

,object,name,lang,released_at,layout,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,legalities,reserved,foil,nonfoil,promo,reprint,variation,set,set_name,set_type,rarity,artist,border_color,frame,full_art,textless,booster,card_faces,promo_types,edhrec_rank,loyalty
56,card,Lovestruck Beast // Heart's Desire,en,2019-10-04,adventure,{2}{G} // {G},3.0,Creature — Beast Noble // Sorcery — Adventure,NaN,5,5,[G],[G],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,False,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Kev Walker,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Lovestruck B...","[prerelease, datestamped]",13909.0,NaN
57,card,Lovestruck Beast // Heart's Desire,en,2019-10-04,adventure,{2}{G} // {G},3.0,Creature — Beast Noble // Sorcery — Adventure,NaN,5,5,[G],[G],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,True,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Kev Walker,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Lovestruck B...",NaN,13909.0,NaN
242,card,Lovestruck Beast // Heart's Desire,en,2019-10-04,adventure,{2}{G} // {G},3.0,Creature — Beast Noble // Sorcery — Adventure,NaN,5,5,[G],[G],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,True,True,False,False,eld,Throne of Eldraine,expansion,rare,Tyler Walpole,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Lovestruck B...",NaN,13909.0,NaN
376,card,Lovestruck Beast // Heart's Desire,en,2019-10-04,adventure,{2}{G} // {G},3.0,Creature — Beast Noble // Sorcery — Adventure,NaN,5,5,[G],[G],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,True,False,False,False,eld,Throne of Eldraine,expansion,rare,Kev Walker,black,2015,False,False,True,"[{'object': 'card_face', 'name': 'Lovestruck B...",NaN,13909.0,NaN


In [50]:
df.isnull().sum()

object                0
name                  0
lang                  0
released_at           0
layout                0
mana_cost           174
cmc                   0
type_line             0
oracle_text         410
power             22247
toughness         22247
colors              174
color_identity        0
legalities            0
reserved              0
foil                  0
nonfoil               0
promo                 0
reprint               0
variation             0
set                   0
set_name              0
set_type              0
rarity                0
artist                0
border_color          0
frame                 0
full_art              0
textless              0
booster               0
card_faces        40852
promo_types       38326
edhrec_rank           0
loyalty           40828
dtype: int64

In [51]:
df.loc[df['card_faces'].notnull()]

,object,name,lang,released_at,layout,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,legalities,reserved,foil,nonfoil,promo,reprint,variation,set,set_name,set_type,rarity,artist,border_color,frame,full_art,textless,booster,card_faces,promo_types,edhrec_rank,loyalty
7,card,Nightmare Moon // Princess Luna,en,2019-10-22,transform,NaN,6.0,Legendary Creature — Alicorn // Legendary Crea...,NaN,NaN,NaN,NaN,[B],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,False,True,False,False,ptg,Ponies: The Galloping,funny,mythic,John Thacker,silver,2015,False,False,False,"[{'object': 'card_face', 'name': 'Nightmare Mo...",NaN,0.0,NaN
56,card,Lovestruck Beast // Heart's Desire,en,2019-10-04,adventure,{2}{G} // {G},3.0,Creature — Beast Noble // Sorcery — Adventure,NaN,5,5,[G],[G],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,False,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Kev Walker,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Lovestruck B...","[prerelease, datestamped]",13909.0,NaN
57,card,Lovestruck Beast // Heart's Desire,en,2019-10-04,adventure,{2}{G} // {G},3.0,Creature — Beast Noble // Sorcery — Adventure,NaN,5,5,[G],[G],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,True,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Kev Walker,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Lovestruck B...",NaN,13909.0,NaN
82,card,Bonecrusher Giant // Stomp,en,2019-10-04,adventure,{2}{R} // {1}{R},3.0,Creature — Giant // Instant — Adventure,NaN,4,3,[R],[R],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,False,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Victor Adame Minguez,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Bonecrusher ...","[prerelease, datestamped]",11665.0,NaN
83,card,Bonecrusher Giant // Stomp,en,2019-10-04,adventure,{2}{R} // {1}{R},3.0,Creature — Giant // Instant — Adventure,NaN,4,3,[R],[R],"{'standard': 'legal', 'future': 'legal', 'hist...",False,True,True,True,True,False,peld,Throne of Eldraine Promos,promo,rare,Victor Adame Minguez,black,2015,False,False,False,"[{'object': 'card_face', 'name': 'Bonecrusher ...",NaN,11665.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37562,card,Wax // Wane,en,2000-10-02,split,{G} // {W},2.0,Instant // Instant,NaN,NaN,NaN,"[G, W]","[G, W]","{'standard': 'not_legal', 'future': 'not_legal...",False,True,True,False,False,False,inv,Invasion,expansion,uncommon,Ben Thompson,black,1997,False,False,True,"[{'object': 'card_face', 'name': 'Wax', 'mana_...",NaN,0.0,NaN
37563,card,Assault // Battery,en,2000-10-02,split,{R} // {3}{G},5.0,Sorcery // Sorcery,NaN,NaN,NaN,"[G, R]","[G, R]","{'standard': 'not_legal', 'future': 'not_legal...",False,True,True,False,False,False,inv,Invasion,expansion,uncommon,Ben Thompson,black,1997,False,False,True,"[{'object': 'card_face', 'name': 'Assault', 'm...",NaN,0.0,NaN
37564,card,Pain // Suffering,en,2000-10-02,split,{B} // {3}{R},5.0,Sorcery // Sorcery,NaN,NaN,NaN,"[B, R]","[B, R]","{'standard': 'not_legal', 'future': 'not_legal...",False,True,True,False,False,False,inv,Invasion,expansion,uncommon,David Martin,black,1997,False,False,True,"[{'object': 'card_face', 'name': 'Pain', 'mana...",NaN,19277.0,NaN
37565,card,Spite // Malice,en,2000-10-02,split,{3}{U} // {3}{B},8.0,Instant // Instant,NaN,NaN,NaN,"[B, U]","[B, U]","{'standard': 'not_legal', 'future': 'not_legal...",False,True,True,False,False,False,inv,Invasion,expansion,uncommon,David Martin,black,1997,False,False,True,"[{'object': 'card_face', 'name': 'Spite', 'man...",NaN,18936.0,NaN


___
Let's clean up our dual cards seperated data before adding it back to the original

In [52]:
dual_cards_sp.loc[dual_cards_sp['colors'].isnull()]

,object,name,mana_cost,type_line,oracle_text,colors,power,toughness,flavor_text,artist,loyalty
0,card_face,Lovestruck Beast,{2}{G},Creature — Beast Noble,Lovestruck Beast can't attack unless you contr...,NaN,5,5,"His mind chose solitude, but his heart disagreed.",Kev Walker,NaN
1,card_face,Heart's Desire,{G},Sorcery — Adventure,Create a 1/1 white Human creature token. (Then...,NaN,NaN,NaN,NaN,Kev Walker,NaN
0,card_face,Lovestruck Beast,{2}{G},Creature — Beast Noble,Lovestruck Beast can't attack unless you contr...,NaN,5,5,"His mind chose solitude, but his heart disagreed.",Kev Walker,NaN
1,card_face,Heart's Desire,{G},Sorcery — Adventure,Create a 1/1 white Human creature token. (Then...,NaN,NaN,NaN,NaN,Kev Walker,NaN
0,card_face,Bonecrusher Giant,{2}{R},Creature — Giant,Whenever Bonecrusher Giant becomes the target ...,NaN,4,3,Not every tale ends in glory.,Victor Adame Minguez,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1,card_face,Suffering,{3}{R},Sorcery,Destroy target land.,NaN,NaN,NaN,NaN,David Martin,NaN
0,card_face,Spite,{3}{U},Instant,Counter target noncreature spell.,NaN,NaN,NaN,NaN,David Martin,NaN
1,card_face,Malice,{3}{B},Instant,Destroy target nonblack creature. It can't be ...,NaN,NaN,NaN,NaN,David Martin,NaN
0,card_face,Stand,{W},Instant,Prevent the next 2 damage that would be dealt ...,NaN,NaN,NaN,NaN,David Martin,NaN


In [53]:
dual_cards_sp.loc[dual_cards_sp['loyalty'].notnull()]

,object,name,mana_cost,type_line,oracle_text,colors,power,toughness,flavor_text,artist,loyalty
1,card_face,"Nicol Bolas, the Arisen",,Legendary Planeswalker — Bolas,"+2: Draw two cards.\n−3: Nicol Bolas, the Aris...","[B, R, U]",NaN,NaN,NaN,Svetlin Velinov,7
1,card_face,"Nicol Bolas, the Arisen",,Legendary Planeswalker — Bolas,"+2: Draw two cards.\n−3: Nicol Bolas, the Aris...","[B, R, U]",NaN,NaN,NaN,Svetlin Velinov,7
1,card_face,"Nissa, Sage Animist",,Legendary Planeswalker — Nissa,+1: Reveal the top card of your library. If it...,[G],NaN,NaN,NaN,Wesley Burt,3
1,card_face,"Liliana, Defiant Necromancer",,Legendary Planeswalker — Liliana,+2: Each player discards a card.\n−X: Return t...,[B],NaN,NaN,NaN,Karla Ortiz,3
1,card_face,"Gideon, Battle-Forged",,Legendary Planeswalker — Gideon,+2: Up to one target creature an opponent cont...,[W],NaN,NaN,NaN,Willian Murai,3
1,card_face,"Jace, Telepath Unbound",,Legendary Planeswalker — Jace,+1: Up to one target creature gets -2/-0 until...,[U],NaN,NaN,NaN,Jaime Jones,5
0,card_face,Garruk Relentless,{3}{G},Legendary Planeswalker — Garruk,When Garruk Relentless has two or fewer loyalt...,[G],NaN,NaN,NaN,Grzegorz Rutkowski,3
1,card_face,"Chandra, Roaring Flame",,Legendary Planeswalker — Chandra,"+1: Chandra, Roaring Flame deals 2 damage to t...",[R],NaN,NaN,NaN,Eric Deschamps,4
0,card_face,Arlinn Kord,{2}{R}{G},Legendary Planeswalker — Arlinn,"+1: Until end of turn, up to one target creatu...","[G, R]",NaN,NaN,NaN,Winona Nelson,3
0,card_face,Arlinn Kord,{2}{R}{G},Legendary Planeswalker — Arlinn,"+1: Until end of turn, up to one target creatu...","[G, R]",NaN,NaN,NaN,Winona Nelson,3
